In [1]:
import torch
import pickle
from uncertainty_rnn import BoardGuesserNet
import torch.optim as optim

In [2]:
# what if we deal with uncertainty with an RNN?
#Treat this like any other machine learning problem X -> Y, except X is obs and Y is truth board

#first we collect data
!python record_data.py random_agent_save_obs random_agent

WHITE's Turn [1]
   A   B   C   D   E   F   G   H  
8|   |   |   |   |   |   |   |   |
7|   |   |   |   |   |   |   |   |
6|   |   |   |   |   |   |   |   |
5|   |   |   |   |   |   |   |   |
4|   |   |   |   |   |   |   |   |
3|   |   |   |   |   |   |   |   |
2| P | P | P | P | P | P | P | P |
1| R | N | B | Q | K | B | N | R |

[WHITE]-- Sense Around Square f1 --
   A   B   C   D   E   F   G   H  
8|   |   |   |   |   |   |   |   |
7|   |   |   |   |   |   |   |   |
6|   |   |   |   |   |   |   |   |
5|   |   |   |   |   |   |   |   |
4|   |   |   |   |   |   |   |   |
3|   |   |   |   |   |   |   |   |
2| P | P | P | P | P | P | P | P |
1| R | N | B | Q | K | B | N | R |

[WHITE]-- Move requested: d2d3 -- Move taken: d2d3
   A   B   C   D   E   F   G   H  
8|   |   |   |   |   |   |   |   |
7|   |   |   |   |   |   |   |   |
6|   |   |   |   |   |   |   |   |
5|   |   |   |   |   |   |   |   |
4|   |   |   |   |   |   |   |   |
3|   |   |   | P |   |   |   |   |
2| P | P | P |   | 

In [3]:
# now lets load the data in
in_ = open("white_game_obs.pkl", "rb")
white_sense_list, white_truth_board_list = pickle.load(in_)
in_.close()

In [4]:
# white sense list is basically a list of one hot encoding of sensing data
print("number of turns in game", white_sense_list[0].shape)
print("dimension of a sense", white_sense_list[0].shape)
# 1st dimension is number of turns, second dimension

number of turns in game (20, 8, 8)
dimension of a sense (20, 8, 8)


In [5]:
#load the recurrent network
guessNet = BoardGuesserNet()

In [6]:
# prepare data for training
X_train_batch = torch.Tensor(white_sense_list)
y_train_batch = torch.Tensor(white_sense_list)
print(X_train_batch.shape)

torch.Size([23, 20, 8, 8])


In [9]:
#set up loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.SGD(guessNet.parameters(), lr=0.001, momentum=0.9)


In [10]:
# training network on one game. Done
pred_labels = guessNet(X_train_batch)
assert pred_labels.shape == y_train_batch.shape
loss =criterion(pred_labels, y_train_batch)
loss.backward()
optimizer.step() # magic

conv 1 out shape torch.Size([23, 32, 6, 6])
conv 2 out shape torch.Size([23, 64, 5, 5])
pool_2_out shape torch.Size([23, 64, 2, 2])
